<a href="https://colab.research.google.com/github/snklp/computer_vision/blob/master/1_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [67]:
# creating a fully connected layer

class ANN(nn.Module):
  def __init__(self, ip_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(ip_size, 50)
    self.fc2 = nn.Linear(50, 50)
    self.fc3 = nn.Linear(50, 50)
    self.fc4 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return F.log_softmax(x, dim=1)

# model = ANN(784, 10)
# x = torch.randn(64, 784)      ## 64 egs each of size 784
# model(x).shape will be 64,10  ## 64 egs each corresponding to 10 values

In [68]:
# setting up device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [69]:
# Parameters
ip_size = 28*28
num_classes = 10

#Hyperparameters
lr = 0.001
batch_size = 64
num_epochs = 3

In [70]:
# Downloading Data
train_mnist = datasets.MNIST(root='/content/drive/MyDrive/Datasets', train=True, transform=transforms.ToTensor(), download=True)
test_mnist = datasets.MNIST(root='/content/drive/MyDrive/Datasets', train=False, transform=transforms.ToTensor(), download=True)

# Loading Data
train = DataLoader(dataset=train_mnist, batch_size=batch_size, shuffle=True)
test = DataLoader(dataset=test_mnist, batch_size=batch_size, shuffle=True)

In [71]:
# Initialising Network

model = ANN(ip_size=ip_size, num_classes=num_classes).to(device)

In [72]:
# Training Network

for epoch in range(num_epochs):
  for data in train:
    X, y = data
    X = X.to(device=device)
    y = y.to(device=device)

    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])


> **X.shape = (#egs in a batch, (#channels, height, width))**                
Hence we want **(1, 28, 28)** to be flattened to **784**                   
Also since **y** is a **scalar** value Hence we'll use **nll_loss**                  
(if it would be **one hot vector** then we would have used **mean sq. error** loss function)

In [73]:
# Training Network on Train Data

optimizer = optim.Adam(model.parameters(), lr=0.001)  # model.parameters -> changable things about model 

for epoch in range(num_epochs):
  for data in train:
    X, y = data
    X = X.to(device=device)
    y = y.to(device=device)
    model.zero_grad()
    # shape correction according to model
    X = X.reshape(-1, 28*28)

    # forward
    output = model(X)
    loss = F.nll_loss(output, y)
    loss.backward()    # backpropogating loss
    optimizer.step()   # adjusts weights 
    
  print(loss)


tensor(0.1350, grad_fn=<NllLossBackward>)
tensor(0.2337, grad_fn=<NllLossBackward>)
tensor(0.0374, grad_fn=<NllLossBackward>)


In [74]:
# Testing Accuracy on Test Data

num_correct = 0
num_samples = 0

with torch.no_grad():
  for data in test:
    X, y = data
    X = X.to(device=device)
    y = y.to(device=device)
    X = X.reshape(X.shape[0], -1)
    output = model(X)
    for idx, i in enumerate(output):
      if torch.argmax(i) == y[idx]:num_correct += 1
      num_samples += 1

print('Accuracy:',round(num_correct/num_samples, 3))

Accuracy: 0.963
